In [ ]:
# give 2 portforlio, the old one and the optimized new one, 
# calculate the stepwise change of the portfolio
# and prioritize the changes by the improvement of the portfolio

cur_investment = [

        {
            "stock": "GILD",
            "quantity": 55,
        },
        {
            "stock": "LLY",
            "quantity": 6.68,
        },
        {
            "stock": "REGN",
            "quantity": 0.5,
        },
        {
            "stock": "STAN.L",
            "quantity": 196.2
        },
        {
            "stock": "601225.SS",
            "quantity": 600
        },
        {
            "stock": "600276.SS",
            "quantity": 300
        },
        {
            "stock": "600900.SS",
            "quantity": 600
        },
        {
            "stock": "600028.SS",
            "quantity": 3300
        },
        {
            "stock": "MRK",
            "quantity": 11.67
        },
        {
            "stock": "WMT",
            "quantity": 37.4
        },
        {
            "stock": "VRTX",
            "quantity": 10.32
        },
        {
          "stock": "LULU",
            "quantity": 7.18
        }
    ]


In [ ]:
# compute the portfolio value

from util import get_currency_pair, read_and_filter_exchange_rates

tot_value = 0
for stock in cur_investment:
    # get the price of the stock which is the last line in the stock price file
    file_path = './data/prices/' + stock["stock"] + '.csv'
    with open(file_path) as f:
        lines = f.readlines()
        stock_price = float(lines[-1].split(",")[1])
        # if the price is zero, use the previous price
        if stock_price == 0:
            stock_price = float(lines[-2].split(",")[1])

    if '.' in stock["stock"]:
        stock_suffix = '.' + stock["stock"].split(".")[-1]
        # convert GBX to GBP
        if stock_suffix == '.L':
            stock_price *= 0.01
        exchange_name, needs_inversion, exchange_name_yahoo = get_currency_pair(stock_suffix, "USD")
        df_rate = read_and_filter_exchange_rates(exchange_name, exchange_name_yahoo)
        #print(df_rate)
        rate = df_rate[exchange_name].iloc[-1]

        if needs_inversion:
            rate = 1 / rate

        print(stock["stock"], stock_price, stock_price * rate, stock["quantity"] * stock_price * rate)
        tot_value += stock["quantity"] * stock_price * rate
        stock['value'] = stock["quantity"] * stock_price * rate
    else:
        print(stock["stock"], stock_price, stock["quantity"] * stock_price)
        tot_value += stock["quantity"] * stock_price
        stock['value'] = stock["quantity"] * stock_price


print(tot_value)


In [ ]:
# compute the weight
for stock in cur_investment:
    stock['weight'] = stock['value'] / tot_value

cur_investment

In [ ]:
import pandas as pd
import numpy as np
import json

# enumerate the directory and find out the latest file
import os


data_dir = "./processed_data_128"

S = pd.read_pickle(f'{data_dir}/S.pkl')
mu = np.load(f'{data_dir}/mu.npy')
# read the latest json data


latest_file = None
latest_time = 0
for file in os.listdir(data_dir + '/computed_portfolios'):
    if file.endswith(".json"):
        file_time = os.path.getmtime(data_dir + '/computed_portfolios/' + file)
        if file_time > latest_time:
            latest_time = file_time
            latest_file = file

with open(data_dir + '/computed_portfolios/' + latest_file) as f:
    tickers_to_buy = json.load(f)

with open(f'{data_dir}/valid_tickers.txt', 'r') as f:
    valid_tickers = f.read().splitlines()


In [ ]:
current_weight = np.zeros(len(valid_tickers))
for stock in cur_investment:
    if stock["stock"] in valid_tickers:
        current_weight[valid_tickers.index(stock["stock"])] = stock["weight"]

In [ ]:
import sys


from util import portfolio_log_return, portfolio_return, portfolio_volatility, portfolio_volatility_log_return


print(portfolio_volatility(current_weight, S))
print(portfolio_return(current_weight, mu))
sharpe = portfolio_log_return(current_weight, mu) - portfolio_volatility_log_return(current_weight, S)
print(sharpe)

In [ ]:
tickers_to_buy

In [ ]:
def update_weight(weights_attr, unique_ids):
  weights = np.zeros(len(unique_ids))
  for asset in weights_attr:
    asset_id = asset['id']
    asset_weight = asset['weight']
    asset_index = unique_ids.index(asset_id)
    weights[asset_index] = asset_weight
  return weights

In [ ]:
tickers_to_buy

In [ ]:
opti_weight = update_weight(tickers_to_buy, valid_tickers)

print(portfolio_volatility(opti_weight, S))
print(portfolio_return(opti_weight, mu))
sharpe = portfolio_log_return(opti_weight, mu) - portfolio_volatility_log_return(opti_weight, S)
print(sharpe)

In [ ]:
# to find the steps to optimize my current portfolio to the optimized portfolio
# we need to find the difference between the optimized portfolio and the current portfolio
# and then prioritize the changes by the improvement of the portfolio

old_portfolio = pd.DataFrame(cur_investment)
optimized_portfolio = pd.DataFrame(tickers_to_buy)

# Merge the two dataframes to compare weights
comparison = pd.merge(old_portfolio, optimized_portfolio, left_on='stock', right_on='id', how='outer')
comparison['stock'] = comparison['stock'].fillna(comparison['id'])
comparison.fillna(0, inplace=True)  # Assuming no stock means 0 weight
comparison['weight_difference'] = comparison['weight_y'] - comparison['weight_x']

#*comparison['new_quantity'] = comparison['quantity'] * comparison['weight_y'] / comparison['weight_x'] if comparison['weight_x'] != 0 else 0
print(comparison[['stock','weight_x', 'weight_y', 'weight_difference', 'quantity']])

In [ ]:
comparison

In [ ]:
def portfolio_volatility_log_return(weights, covariance):
    return np.sqrt(np.dot(weights.T, np.dot(covariance, weights)))

def portfolio_var_log_return(weights, covariance):
    return np.dot(weights.T, np.dot(covariance, weights))

def portfolio_log_return(weights, returns, allow_short=False):
    return np.sum(np.abs(returns)*weights) if allow_short else np.sum(returns*weights)

In [ ]:
# Initialize the weights array

current_weight = np.zeros(len(valid_tickers))
for stock in cur_investment:
    if stock["stock"] in valid_tickers:
        current_weight[valid_tickers.index(stock["stock"])] = stock["weight"]


original_sharpe = portfolio_log_return(current_weight, mu) - portfolio_volatility_log_return(current_weight, S)
print(f'Original Sharpe ratio: {original_sharpe}')
# Optimize one stock at a time and adjust other weights proportionally

new_weights = current_weight.copy()
for i in range(20):
    results = []
    print(f'Starting iteration {i} optimization...')
    for index, row in comparison.iterrows():
        new_weights_try = new_weights.copy()
        stock_name = comparison.loc[index, 'stock']
        stock_idx = valid_tickers.index(stock_name)
        # Adjust the targeted stock weight
        new_weights_try[stock_idx] = comparison.loc[index, 'weight_y']
        # Adjust the rest to ensure total sum remains 1
        total_adjusted_weights = np.sum(new_weights_try) - new_weights_try[stock_idx]
        scale_factor = (1 - new_weights_try[stock_idx]) / total_adjusted_weights
        for j in range(len(new_weights_try)):
            if j != stock_idx:
                new_weights_try[j] *= scale_factor

        new_sharpe = portfolio_log_return(new_weights_try, mu) - portfolio_volatility_log_return(new_weights_try, S)
        results.append((comparison.loc[index, 'stock'], new_sharpe, new_sharpe - original_sharpe, new_weights_try, comparison.loc[index, 'weight_difference']))

    results.sort(key=lambda x: x[2], reverse=True)

    stock_name = results[0][0]
    stock_idx = valid_tickers.index(stock_name)
    # Adjust the targeted stock weight
    new_weights = results[0][3]
    new_sharpe = results[0][1]
    # remove the row from comparison
    comparison = comparison[comparison['stock'] != stock_name]

    original_sharpe = new_sharpe

    print(f'Iteration {i} optimization result:')
    print(f'Optimized ticker: {stock_name}')
    print(f'Sharpe ratio: {results[0][1]}')
    print(f'Improvement: {results[0][2]}')
    print(f'Weight difference: {results[0][4]}')
    


In [ ]:
optimized_portfolio